Amazon content based recommendation system

In [61]:
#import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import io

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#loading dataset
data = pd.read_json('/content/drive/My Drive/amazon recommendation system/tops_fashion.json')

Basic dataframe info

In [3]:
data.shape

(183138, 19)

In [4]:
data.head()

,sku,asin,product_type_name,formatted_price,author,color,brand,publisher,availability,reviews,large_image_url,availability_type,small_image_url,editorial_review,title,model,medium_image_url,manufacturer,editorial_reivew
0,None,B016I2TS4W,SHIRT,None,None,None,FNC7C,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Women's O Nec...,Minions Como Superheroes Ironman Long Sleeve R...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
1,None,B01N49AI08,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Izo Tunic,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
2,None,B01JDPCOHO,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Won Top,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
3,None,B01N19U5H5,SHIRT,None,None,None,Focal18,None,None,"[True, https://www.amazon.com/reviews/iframe?a...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,100% Brand New & Fashion<br> Quantity: 1 Piece...,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
4,None,B004GSI2OS,SHIRT,$26.26,None,Onyx Black/ Stone,FeatherLite,None,Usually ships in 6-10 business days,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,now,https://images-na.ssl-images-amazon.com/images...,,Featherlite Ladies' Long Sleeve Stain Resistan...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183138 entries, 0 to 183137
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   sku                363 non-null     object
 1   asin               183138 non-null  object
 2   product_type_name  183138 non-null  object
 3   formatted_price    28395 non-null   object
 4   author             1 non-null       object
 5   color              64956 non-null   object
 6   brand              182987 non-null  object
 7   publisher          42899 non-null   object
 8   availability       24532 non-null   object
 9   reviews            183138 non-null  object
 10  large_image_url    183138 non-null  object
 11  availability_type  24559 non-null   object
 12  small_image_url    183138 non-null  object
 13  editorial_review   2758 non-null    object
 14  title              183138 non-null  object
 15  model              62370 non-null   object
 16  medium_image_url   1

In [6]:
data.columns

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')

Taking only 7 features for further process

*   asin(unique key to each of the products)
*   brand
*   color
*   product type name
*   medium image url
*   title
*   formatted price






In [7]:
data = data[['asin', 'brand', 'color', 'product_type_name', 'medium_image_url', 'title', 'formatted_price']]

In [8]:
data.shape

(183138, 7)

Data cleanning and understanding

In [9]:
data.describe()

,asin,brand,color,product_type_name,medium_image_url,title,formatted_price
count,183138,182987,64956,183138,183138,183138,28395
unique,183138,10577,7380,72,170782,175985,3135
top,B00URJ1XS2,Zago,Black,SHIRT,https://images-na.ssl-images-amazon.com/images...,Nakoda Cotton Self Print Straight Kurti For Women,$19.99
freq,1,223,13207,167794,23,77,945


In [10]:
data.isnull().sum()

asin                      0
brand                   151
color                118182
product_type_name         0
medium_image_url          0
title                     0
formatted_price      154743
dtype: int64

In [11]:
data['product_type_name'].unique()

array(['SHIRT', 'SWEATER', 'APPAREL', 'OUTDOOR_RECREATION_PRODUCT',
       'BOOKS_1973_AND_LATER', 'PANTS', 'HAT', 'SPORTING_GOODS', 'DRESS',
       'UNDERWEAR', 'SKIRT', 'OUTERWEAR', 'BRA', 'ACCESSORY',
       'ART_SUPPLIES', 'SLEEPWEAR', 'ORCA_SHIRT', 'HANDBAG',
       'PET_SUPPLIES', 'SHOES', 'KITCHEN', 'ADULT_COSTUME',
       'HOME_BED_AND_BATH', 'MISC_OTHER', 'BLAZER',
       'HEALTH_PERSONAL_CARE', 'TOYS_AND_GAMES', 'SWIMWEAR',
       'CONSUMER_ELECTRONICS', 'SHORTS', 'HOME', 'AUTO_PART',
       'OFFICE_PRODUCTS', 'ETHNIC_WEAR', 'BEAUTY',
       'INSTRUMENT_PARTS_AND_ACCESSORIES', 'POWERSPORTS_PROTECTIVE_GEAR',
       'SHIRTS', 'ABIS_APPAREL', 'AUTO_ACCESSORY', 'NONAPPARELMISC',
       'TOOLS', 'BABY_PRODUCT', 'SOCKSHOSIERY',
       'POWERSPORTS_RIDING_SHIRT', 'EYEWEAR', 'SUIT', 'OUTDOOR_LIVING',
       'POWERSPORTS_RIDING_JACKET', 'HARDWARE', 'SAFETY_SUPPLY',
       'ABIS_DVD', 'VIDEO_DVD', 'GOLF_CLUB', 'MUSIC_POPULAR_VINYL',
       'HOME_FURNITURE_AND_DECOR', 'TABLET_COMPUTER',

product type name

In [12]:
data['product_type_name'].describe()

count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object

In [13]:
#most frequent product type name
most_freq_product = Counter(list(data['product_type_name']))
most_freq_product.most_common(10)

[('SHIRT', 167794),
 ('APPAREL', 3549),
 ('BOOKS_1973_AND_LATER', 3336),
 ('DRESS', 1584),
 ('SPORTING_GOODS', 1281),
 ('SWEATER', 837),
 ('OUTERWEAR', 796),
 ('OUTDOOR_RECREATION_PRODUCT', 729),
 ('ACCESSORY', 636),
 ('UNDERWEAR', 425)]

In [14]:
data['product_type_name'].isnull().sum()

0

color

In [15]:
data['color'].describe()

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object

In [16]:
most_freq_color = Counter(list(data['color']))
most_freq_color.most_common(10)

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

In [17]:
data['color'].isnull().sum()

118182

Formatted price

In [18]:
data['formatted_price'].describe()

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object

In [19]:
most_freq_price = Counter(list(data['formatted_price']))
most_freq_price.most_common(10)

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

In [20]:
data['formatted_price'].isnull().sum()

154743

Brand

In [21]:
data['brand'].describe()

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object

In [22]:
most_freq_brand = Counter(list(data['brand']))
most_freq_brand.most_common(10)

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

In [23]:
data['brand'].isnull().sum()

151

Title

In [24]:
data['title'].describe()

count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object

In [25]:
data['title'].isnull().sum()

0

Removing some datapoints which are null to reduce time and space complexity.

In [26]:
data = data.loc[~data['formatted_price'].isnull()]#removes formatted price is null
data = data.loc[~data['color'].isnull()]#removes color is null
data = data.loc[~data['brand'].isnull()]#removes brand is null

In [27]:
data.shape

(28292, 7)

In [28]:
data.isnull().sum()

asin                 0
brand                0
color                0
product_type_name    0
medium_image_url     0
title                0
formatted_price      0
dtype: int64

removing duplicates datapoints

In [29]:
#no of datapoints with same title
sum(data.duplicated('title'))

2323

In [30]:
#print all datapoints with fewer title len
data[data['title'].apply(lambda x: len(x.split())<4)].shape

(213, 7)

In [31]:
#remove datapoints with fewer title length
data = data[data['title'].apply(lambda x:len(x.split())>4)]

In [32]:
data.shape

(27858, 7)

In [33]:
#sort dataset based on title
data_sorted = data.sort_values('title', ascending=False)

In [34]:
data_sorted.head()

,asin,brand,color,product_type_name,medium_image_url,title,formatted_price
61973,B06Y1KZ2WB,Éclair,Black/Pink,SHIRT,https://images-na.ssl-images-amazon.com/images...,Éclair Women's Printed Thin Strap Blouse Black...,$24.99
133820,B010RV33VE,xiaoming,Pink,SHIRT,https://images-na.ssl-images-amazon.com/images...,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19
81461,B01DDSDLNS,xiaoming,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,xiaoming Women's White Long Sleeve Single Brea...,$21.58
75995,B00X5LYO9Y,xiaoming,Red Anchors,SHIRT,https://images-na.ssl-images-amazon.com/images...,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91
151570,B00WPJG35K,xiaoming,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32


some of the title for products are almost same with fewer different in the word.


> these titles are same for same product with different color and sizes.



In [35]:
#removing datapoints with almost same titles
#this takes long time to run so save cleanned data and load directly
'''indices = []
for i,row in data_sorted.iterrows():
    indices.append(i)


import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
    
    previous_i = i

    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = data['title'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:

        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = data['title'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))
        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break'''

"indices = []\nfor i,row in data_sorted.iterrows():\n    indices.append(i)\n\n\nimport itertools\nstage1_dedupe_asins = []\ni = 0\nj = 0\nnum_data_points = data_sorted.shape[0]\nwhile i < num_data_points and j < num_data_points:\n    \n    previous_i = i\n\n    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']\n    a = data['title'].loc[indices[i]].split()\n\n    # search for the similar products sequentially \n    j = i+1\n    while j < num_data_points:\n\n        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']\n        b = data['title'].loc[indices[j]].split()\n\n        # store the maximum length of two strings\n        length = max(len(a), len(b))\n        # count is used to store the number of words that are matched in both strings\n        count  = 0\n\n        # itertools.zip_longest(a,b): will map the corresponding w

In [36]:
'''data = data.loc[data['asin'].isin(stage1_dedupe_asins)]'''

"data = data.loc[data['asin'].isin(stage1_dedupe_asins)]"

In [37]:
'''indices = []
for i,row in data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(data['asin'].loc[i])
    # consider the first apperal's title
    a = data['title'].loc[i].split()
    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    for j in indices:
        
        b = data['title'].loc[j].split()
        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
        
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        # if the number of words in which both strings differ are < 3 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)'''

"indices = []\nfor i,row in data.iterrows():\n    indices.append(i)\n\nstage2_dedupe_asins = []\nwhile len(indices)!=0:\n    i = indices.pop()\n    stage2_dedupe_asins.append(data['asin'].loc[i])\n    # consider the first apperal's title\n    a = data['title'].loc[i].split()\n    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']\n    for j in indices:\n        \n        b = data['title'].loc[j].split()\n        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']\n        \n        length = max(len(a),len(b))\n        \n        # count is used to store the number of words that are matched in both strings\n        count  = 0\n\n        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings\n        # example: a =['a', 'b', 'c', 'd']\n        # b = ['a', 'b'

In [38]:
data.shape

(27858, 7)

In [39]:
#downloading cleanned data to disk
'''from google.colab import files
data.to_csv('cleanned_recommendation_data.csv')
files.download('cleanned_recommendation_data.csv')'''

"from google.colab import files\ndata.to_csv('cleanned_recommendation_data.csv')\nfiles.download('cleanned_recommendation_data.csv')"

In [40]:
#loadding cleaned dataframe from google drive
data = pd.read_csv('/content/drive/My Drive/amazon recommendation system/cleanned_recommendation_data.csv')

In [41]:
data.shape

(16388, 8)

In [42]:
data.head()

,Unnamed: 0,asin,brand,color,product_type_name,medium_image_url,title,formatted_price
0,4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,SHIRT,https://images-na.ssl-images-amazon.com/images...,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26
1,6,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,Women's Unique 100% Cotton T - Special Olympic...,$9.99
2,15,B003BSRPB0,FeatherLite,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,FeatherLite Ladies' Moisture Free Mesh Sport S...,$20.54
3,27,B014ICEJ1Q,FNC7C,Purple,SHIRT,https://images-na.ssl-images-amazon.com/images...,Supernatural Chibis Sam Dean And Castiel O Nec...,$7.39
4,46,B01NACPBG2,Fifth Degree,Black,SHIRT,https://images-na.ssl-images-amazon.com/images...,Fifth Degree Womens Gold Foil Graphic Tees Jun...,$6.95


In [43]:
data = data[['asin', 'brand', 'color', 'product_type_name', 'medium_image_url', 'title', 'formatted_price']]

In [44]:
data.head()

,asin,brand,color,product_type_name,medium_image_url,title,formatted_price
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,SHIRT,https://images-na.ssl-images-amazon.com/images...,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,Women's Unique 100% Cotton T - Special Olympic...,$9.99
2,B003BSRPB0,FeatherLite,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,FeatherLite Ladies' Moisture Free Mesh Sport S...,$20.54
3,B014ICEJ1Q,FNC7C,Purple,SHIRT,https://images-na.ssl-images-amazon.com/images...,Supernatural Chibis Sam Dean And Castiel O Nec...,$7.39
4,B01NACPBG2,Fifth Degree,Black,SHIRT,https://images-na.ssl-images-amazon.com/images...,Fifth Degree Womens Gold Foil Graphic Tees Jun...,$6.95


Text preprocessing

*   removing punctuation
*   removing stopwords
*   converting to lowercase
*   performing steamming





In [45]:
stopword = set(stopwords.words('english'))#creating set of stopwords
snow = SnowballStemmer('english')#initialization of snowballstemming
#removing some words from stopwords which are useful in title
ls = ['o', 's', 'm', 't']
for i in ls:
  stopword.remove(i)


In [46]:
start = time.clock()
clean_titles = []
for title in data['title'].values:
  clean_words = []
  punc_free =  re.sub(r'[^A-Za-z0-9+.\-]+', ' ', title)#removing punctuation
  for word in punc_free.split():
    lower = word.lower()
    if lower not in stopword:
      s = snow.stem(lower)
      clean_words.append(s)
  clean_title = ' '.join(clean_words) 
  clean_titles.append(clean_title)     
print(time.clock()-start)



2.070373


In [47]:
len(clean_titles)

16388

In [48]:
#adding clean_title to dataframe
data['clean_title'] = clean_titles
#removing old title with not cleanned
clean_data = data.drop(['title'], axis=1)

In [49]:
clean_data.head()

,asin,brand,color,product_type_name,medium_image_url,formatted_price,clean_title
0,B004GSI2OS,FeatherLite,Onyx Black/ Stone,SHIRT,https://images-na.ssl-images-amazon.com/images...,$26.26,featherlit ladi long sleev stain resist taper ...
1,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,$9.99,women s uniqu 100 cotton t - special olymp wor...
2,B003BSRPB0,FeatherLite,White,SHIRT,https://images-na.ssl-images-amazon.com/images...,$20.54,featherlit ladi moistur free mesh sport shirt ...
3,B014ICEJ1Q,FNC7C,Purple,SHIRT,https://images-na.ssl-images-amazon.com/images...,$7.39,supernatur chibi sam dean castiel o neck t-shi...
4,B01NACPBG2,Fifth Degree,Black,SHIRT,https://images-na.ssl-images-amazon.com/images...,$6.95,fifth degre women gold foil graphic tee junior...


Text based similarity

In [50]:
#import necessary libararies for images
from io import BytesIO
import requests
from PIL import Image

In [51]:
#function to show image
def display_img(url):
    # we get the url of the apparel and download it
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    # we will display it in notebook 
    plt.imshow(img)
    plt.show()

1.Bag of word(BOW) based

In [52]:
#applying bag of word to title
bow = CountVectorizer()
title_feat = bow.fit_transform(clean_data['clean_title'])
title_feat.shape

(16388, 10092)

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#function to give recommendation to query title
def bow_recommendation(index, number):
  pairwise_dist_matrix = pairwise_distances(title_feat, title_feat[index])#calculates distance between title feat matrix and index array and stores as matrix
  sorted_dist_index = np.argsort(pairwise_dist_matrix.flatten())[0:number]#stores the indeces of sorted distance
  sorted_dist_value = np.sort(pairwise_dist_matrix.flatten())[0:number]#stores the values of sorted distance
  df_indices = list(data.index[sorted_dist_index])#list of index of recommended items
  for i in range(0,len(sorted_dist_index)):
        print('ASIN :',data['asin'].loc[df_indices[i]])
        print ('Brand:', data['brand'].loc[df_indices[i]])
        print ('Title:', data['title'].loc[df_indices[i]])
        print ('Euclidean similarity with the query image :', sorted_dist_value[i])
        display_img(data['medium_image_url'].iloc[sorted_dist_index[i]])
        print('='*60)

In [55]:
#bow_recommendation(2,4)#we can check recommended items by passing index of datapoint and number of items to recommend

TFIDF based product similarity

In [56]:
tfidf = TfidfVectorizer()
tfidf_title = tfidf.fit_transform(clean_data['clean_title'])

In [57]:
tfidf_title.shape

(16388, 10092)

In [58]:
#function to recommend similar items
def tfidf_recommendation(index, number):
  pairwise_dist_matrix = pairwise_distances(tfidf_title, tfidf_title[index])
  sorted_dist_index = np.argsort(pairwise_dist_matrix.flatten())[0:number]
  sorted_dist_value = np.sort(pairwise_dist_matrix.flatten())[0:number]
  df_index = list(data.index[sorted_dist_index])
  for i in np.arange(0, len(sorted_dist_index)):
    print('asin:', data['asin'].iloc[sorted_dist_index[i]])
    print('brand:', data['brand'].iloc[sorted_dist_index[i]])
    print('title:', data['title'].iloc[sorted_dist_index[i]])
    print('euclidean distance:', sorted_dist_value[i])
    display_img(data['medium_image_url'].iloc[sorted_dist_index[i]])
    print('-'*50)

In [59]:
#tfidf_recommendation(12566,5)

Conclusion:

Tfidf is better solution to recommend the similar product than bag of word.




